In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/y_duple_filtered_up_0716.csv', encoding='utf-8-sig')

In [4]:
import pandas as pd
from konlpy.tag import Okt

# 1. 간단한 감성 단어 리스트 직접 정의 (임시용)
positive_keywords = ['좋다', '깨끗', '편안', '친절', '만족', '훌륭', '예쁘', '쾌적', '최고', '넓', '추천', '청결', '훌륭', '서비스']
negative_keywords = ['시끄럽', '지저분', '불친절', '좁', '비싸', '실망', '냄새', '불편', '최악', '더럽', '아쉽', '아쉬운']

# 2. 형태소 분석기
okt = Okt()

# 3. 리뷰 → 감성 라벨링 함수
def classify_sentiment(text):
    tokens = okt.morphs(text)
    pos = sum(1 for token in tokens if any(p in token for p in positive_keywords))
    neg = sum(1 for token in tokens if any(n in token for n in negative_keywords))

    if pos > neg:
        return '긍정'
    elif neg > pos:
        return '부정'
    elif pos == neg and pos > 0:
        return '중립'
    else:
        return '중립'

# 4. 데이터 불러오기
df['clean_reviews'] = df['clean_reviews'].astype(str)

# 5. 감성 라벨 생성
df['emotion'] = df['clean_reviews'].apply(classify_sentiment)

In [ ]:
group = df.groupby('name')['감성점수']
positives = group.apply(lambda x: (x == 1).sum())
total = group.count()
긍정률 = positives / total

In [ ]:
# 등급 부여 함수 정의
def get_emotion_grade(score):
    if score > 0.6:
        return "매우 긍정적"
    elif score > 0.3:
        return "긍정적"
    elif score > 0:
        return "약간 긍정"
    elif score == 0:
        return "중립"
    elif score > -0.3:
        return "약간 부정"
    else:
        return "부정적"

# 평균 점수 기반 감성 등급 계산
df['평균감성점수'] = df.groupby('숙소명')['감성점수'].transform('mean')
df['감성등급'] = df['평균감성점수'].apply(get_emotion_grade)

In [ ]:
columns = ['숙소명', '리뷰내용', '감성라벨', '감성점수', '평균감성점수', '감성등급']
df[columns].head()